In [1]:
# Segmentation steps: (a) Original, (b) thresholding, (c) background
# removal, (d) airway and noise removal, (e) rolling–ball operator, (f) left, right
# lung separated.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

import SimpleITK as sitk

# Download data to work on
# %run update_path_to_download_script
# from downloaddata import fetch_data as fdata
from myshow import myshow, myshow3d

In [3]:
# import dicom2nifti
# import os

# output_path = "C:/Users/edwin/OneDrive/Documents/CSC-821/GroupProject/SimpleITK-Notebooks/Python/patient_nifti_data"
# one_patient_path = "C:/Users/edwin/OneDrive/Documents/CSC-821/ImageJAndData/CovidData/manifest-1608266677008/MIDRC-RICORD-1A/MIDRC-RICORD-1A-419639-000831/01-14-2003-NA-CT CHEST PULMONARY EMBOLISM CTPE-44168/2.000000-PE Smart Prep Left Atrium-34513"
# dicom2nifti.convert_dicom.dicom_series_to_nifti(one_patient_path, os.path.join(output_path, 'patient_test_1.nii'))


In [4]:
# img_T1 = sitk.ReadImage("C:/Users/edwin/OneDrive/Documents/CSC-821/ImageJAndData/CovidData/manifest-1608266677008/MIDRC-RICORD-1A/MIDRC-RICORD-1A-419639-000831/01-14-2003-NA-CT CHEST PULMONARY EMBOLISM CTPE-44168/2.000000-PE Smart Prep Left Atrium-34513/1-003.dcm")
# img_T1 = sitk.ReadImage("C:/Users/edwin/OneDrive/Documents/CSC-821/ImageJAndData/CovidData/manifest-1608266677008/MIDRC-RICORD-1A/MIDRC-RICORD-1A-419639-000831/01-14-2003-NA-CT CHEST PULMONARY EMBOLISM CTPE-44168/1.000000-PE SCOUT-66698/1-1.dcm")

img_T1 = sitk.ReadImage("C:/Users/edwin/OneDrive/Documents/CSC-821/GroupProject/SimpleITK-Notebooks/Python/patient_nifti_data/patient_test_1.nii")

img_T1_255 = sitk.Cast(sitk.RescaleIntensity(img_T1), sitk.sitkUInt8)

myshow3d(img_T1)


interactive(children=(IntSlider(value=112, description='z', max=225), Output()), _dom_classes=('widget-interac…

In [5]:
# Edge preserving smoothing
# smooth = sitk.BilateralImageFilter()
# img_T1 = smooth.Execute(img_T1)
# myshow3d(img_T1)

In [6]:
# Open on expternal viewer
# setx SITK_SHOW_COMMAND "C:\Users\edwin\OneDrive\Documents\CSC-821\Fiji\fiji-win64\Fiji.app\ImageJ-win64.exe"
%env SITK_SHOW_COMMAND "C:\Users\edwin\OneDrive\Documents\CSC-821\Fiji\fiji-win64\Fiji.app\ImageJ-win64.exe"
# %env SITK_SHOW_COMMAND "C:/Users/edwin/OneDrive/Documents/CSC-821/Fiji/fiji-win64/Fiji.app/ImageJ-win64.exe"


# sitk.Show(img_T1)

env: SITK_SHOW_COMMAND="C:\Users\edwin\OneDrive\Documents\CSC-821\Fiji\fiji-win64\Fiji.app\ImageJ-win64.exe"


In [7]:
# seg = sitk.BinaryThreshold(
#     img_T1, lowerThreshold=100, upperThreshold=400, insideValue=1, outsideValue=0
# )
seg = sitk.BinaryThreshold(
    img_T1, lowerThreshold=-850, upperThreshold=-200, insideValue=1, outsideValue=0
)

myshow(sitk.LabelOverlay(img_T1_255, seg), "Binary Thresholding overlayed on Image")
myshow(seg, "Binary Thresholding")

interactive(children=(IntSlider(value=112, description='z', max=225), Output()), _dom_classes=('widget-interac…

interactive(children=(IntSlider(value=112, description='z', max=225), Output()), _dom_classes=('widget-interac…

In [8]:
otsu_filter = sitk.OtsuThresholdImageFilter()
otsu_filter.SetInsideValue(0)
otsu_filter.SetOutsideValue(1)
seg = otsu_filter.Execute(img_T1)
myshow(sitk.LabelOverlay(img_T1_255, seg), "Otsu Thresholding")

print(otsu_filter.GetThreshold())

interactive(children=(IntSlider(value=112, description='z', max=225), Output()), _dom_classes=('widget-interac…

-2976.0


In [9]:
otsu_filter = sitk.OtsuMultipleThresholdsImageFilter()
otsu_filter.SetNumberOfThresholds(2)
seg = otsu_filter.Execute(img_T1)
myshow(sitk.LabelOverlay(img_T1_255, seg), "Otsu Multi Thresholding overlayed on Image")
myshow(seg, "Otsu Multi Thresholding")
# sitk.Show(sitk.LabelOverlay(img_T1_255, seg))
print(otsu_filter.GetThresholds())

interactive(children=(IntSlider(value=112, description='z', max=225), Output()), _dom_classes=('widget-interac…

interactive(children=(IntSlider(value=112, description='z', max=225), Output()), _dom_classes=('widget-interac…

(-2976.379093170166, -452.470947265625)


In [10]:
cleaned_thresh_img = sitk.BinaryOpeningByReconstruction(seg, [10, 10, 10])
cleaned_thresh_img = sitk.BinaryClosingByReconstruction(
    cleaned_thresh_img, [10, 10, 10]
)

myshow(sitk.LabelOverlay(img_T1_255, cleaned_thresh_img), "Otsu Multi Thresholding + Bianry Reconstruction Overlayed On Image")


interactive(children=(IntSlider(value=112, description='z', max=225), Output()), _dom_classes=('widget-interac…

In [12]:
seed_pts = [(125,237,200), (369,237,200)]
temp_img = sitk.ConfidenceConnected(img_T1, seedList=seed_pts,
                                                           numberOfIterations=0,
                                                           multiplier=2,
                                                           initialNeighborhoodRadius=1,
                                                           replaceValue=1)

In [13]:
# gui.MultiImageDisplay(image_list=[sitk.LabelOverlay(self.img_uint8, self.temp_img)],
#                               title_list=[title])

In [15]:
# Watershed segmentation
closing = sitk.BinaryMorphologicalClosingImageFilter()
closing.SetForegroundValue(1)
closing.SetKernelRadius(7)
temp_img2 = closing.Execute(temp_img)

myshow(sitk.LabelOverlay(img_T1_255, temp_img2), "Watershed Segmentation Overlayed On Image")



interactive(children=(IntSlider(value=112, description='z', max=225), Output()), _dom_classes=('widget-interac…

In [ ]:


# %matplotlib notebook
# import gui
# import matplotlib.pyplot as plt

# import numpy as np
# from scipy import linalg

# from ipywidgets import interact, fixed

In [ ]:
# spherical_fiducials_image = sitk.ReadImage("C:/Users/edwin/OneDrive/Documents/CSC-821/GroupProject/SimpleITK-Notebooks/Python/patient_nifti_data/patient_test_1.nii")

# roi_list = [((280, 320), (65, 90), (8, 30)), ((200, 240), (65, 100), (15, 40))]


In [ ]:
# roi_acquisition_interface = gui.ROIDataAquisition(spherical_fiducials_image)
# roi_acquisition_interface.set_rois(roi_list)

In [ ]:
# specified_rois = roi_acquisition_interface.get_rois()
# # select the one ROI we will work on
# ROI_INDEX = 0

# roi = specified_rois[ROI_INDEX]
# mask_value = 255

# mask = sitk.Image(spherical_fiducials_image.GetSize(), sitk.sitkUInt8)
# mask.CopyInformation(spherical_fiducials_image)
# mask[
#     roi[0][0] : roi[0][1] + 1, roi[1][0] : roi[1][1] + 1, roi[2][0] : roi[2][1] + 1
# ] = mask_value

In [ ]:
# intensity_values = sitk.GetArrayViewFromImage(spherical_fiducials_image)
# roi_intensity_values = intensity_values[
#     roi[2][0] : roi[2][1], roi[1][0] : roi[1][1], roi[0][0] : roi[0][1]
# ].flatten()
# plt.figure()
# plt.hist(roi_intensity_values, bins=100)
# plt.title("Intensity Values in ROI")
# plt.show()

In [ ]:
# # Set pixels that are in [min_intensity,otsu_threshold] to inside_value, values above otsu_threshold are
# # set to outside_value. The sphere's have higher intensity values than the background, so they are outside.

# inside_value = 0
# outside_value = 255
# number_of_histogram_bins = 100
# mask_output = True

# labeled_result = sitk.OtsuThreshold(
#     spherical_fiducials_image,
#     mask,
#     inside_value,
#     outside_value,
#     number_of_histogram_bins,
#     mask_output,
#     mask_value,
# )

# # Estimate the sphere radius from the segmented image using the LabelShapeStatisticsImageFilter.
# label_shape_analysis = sitk.LabelShapeStatisticsImageFilter()
# label_shape_analysis.SetBackgroundValue(inside_value)
# label_shape_analysis.Execute(labeled_result)
# print(
#     "The sphere's location is: {0:.2f}, {1:.2f}, {2:.2f}".format(
#         *(label_shape_analysis.GetCentroid(outside_value))
#     )
# )
# print(
#     f"The sphere's radius is: {label_shape_analysis.GetEquivalentSphericalRadius(outside_value):.2f}mm"
# )

In [ ]:
# # Visually evaluate the results of segmentation, just to make sure. Use the zoom tool, second from the right, to
# # inspect the segmentation.
# gui.MultiImageDisplay(
#     image_list=[
#         sitk.LabelOverlay(
#             sitk.Cast(
#                 sitk.IntensityWindowing(
#                     spherical_fiducials_image,
#                     windowMinimum=-32767,
#                     windowMaximum=-29611,
#                 ),
#                 sitk.sitkUInt8,
#             ),
#             labeled_result,
#             opacity=0.5,
#         )
#     ],
#     title_list=["thresholding result"],
# );